# Construire un modèle de classification : Délicieuses cuisines asiatiques et indiennes


## Classificateurs de cuisine 2

Dans cette deuxième leçon sur la classification, nous allons explorer `davantage de méthodes` pour classifier les données catégorielles. Nous examinerons également les implications du choix d'un classificateur plutôt qu'un autre.

### [**Quiz avant la leçon**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Prérequis**

Nous supposons que vous avez terminé les leçons précédentes, car nous allons reprendre certains concepts abordés auparavant.

Pour cette leçon, nous aurons besoin des packages suivants :

-   `tidyverse` : Le [tidyverse](https://www.tidyverse.org/) est une [collection de packages R](https://www.tidyverse.org/packages) conçue pour rendre la science des données plus rapide, plus facile et plus agréable !

-   `tidymodels` : Le framework [tidymodels](https://www.tidymodels.org/) est une [collection de packages](https://www.tidymodels.org/packages/) dédiée à la modélisation et à l'apprentissage automatique.

-   `themis` : Le package [themis](https://themis.tidymodels.org/) fournit des étapes supplémentaires pour traiter les données déséquilibrées.

Vous pouvez les installer avec la commande suivante :

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Sinon, le script ci-dessous vérifie si vous avez les packages nécessaires pour compléter ce module et les installe pour vous s'ils sont manquants.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Une carte de classification**

Dans notre [leçon précédente](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), nous avons tenté de répondre à la question : comment choisir entre plusieurs modèles ? En grande partie, cela dépend des caractéristiques des données et du type de problème que nous voulons résoudre (par exemple, classification ou régression ?)

Auparavant, nous avons appris les différentes options disponibles pour classifier des données en utilisant l'aide-mémoire de Microsoft. Le framework de Machine Learning en Python, Scikit-learn, propose un aide-mémoire similaire mais plus détaillé, qui peut vous aider à affiner davantage vos estimateurs (un autre terme pour désigner les classificateurs) :

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Conseil : [consultez cette carte en ligne](https://scikit-learn.org/stable/tutorial/machine_learning_map/) et cliquez sur les chemins pour lire la documentation.  
>  
> Le [site de référence Tidymodels](https://www.tidymodels.org/find/parsnip/#models) propose également une excellente documentation sur les différents types de modèles.

### **Le plan** 🗺️

Cette carte est très utile une fois que vous comprenez bien vos données, car vous pouvez "suivre" ses chemins pour arriver à une décision :

-   Nous avons \>50 échantillons

-   Nous voulons prédire une catégorie

-   Nous avons des données étiquetées

-   Nous avons moins de 100K échantillons

-   ✨ Nous pouvons choisir un Linear SVC

-   Si cela ne fonctionne pas, étant donné que nous avons des données numériques :

    -   Nous pouvons essayer un ✨ KNeighbors Classifier

        -   Si cela ne fonctionne pas, essayez ✨ SVC et ✨ Ensemble Classifiers

C'est un chemin très utile à suivre. Maintenant, passons directement à l'action en utilisant le framework de modélisation [tidymodels](https://www.tidymodels.org/) : une collection cohérente et flexible de packages R développée pour encourager de bonnes pratiques statistiques 😊.

## 2. Diviser les données et gérer un ensemble de données déséquilibré.

Dans nos leçons précédentes, nous avons appris qu'il existait un ensemble d'ingrédients communs à travers nos cuisines. De plus, il y avait une répartition assez inégale du nombre de cuisines.

Nous allons gérer cela en :

-   Supprimant les ingrédients les plus courants qui créent de la confusion entre des cuisines distinctes, en utilisant `dplyr::select()`.

-   Utilisant une `recipe` qui prétraite les données pour les préparer à la modélisation en appliquant un algorithme de `sur-échantillonnage`.

Nous avons déjà vu cela dans la leçon précédente, donc cela devrait être un jeu d'enfant 🥳 !


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Gérer les données déséquilibrées

Les données déséquilibrées ont souvent des effets négatifs sur les performances du modèle. De nombreux modèles fonctionnent mieux lorsque le nombre d'observations est équilibré et, par conséquent, ont tendance à rencontrer des difficultés avec des données non équilibrées.

Il existe principalement deux façons de traiter les ensembles de données déséquilibrés :

-   ajouter des observations à la classe minoritaire : `Sur-échantillonnage`, par exemple en utilisant un algorithme SMOTE qui génère de manière synthétique de nouveaux exemples de la classe minoritaire en utilisant les plus proches voisins de ces cas.

-   supprimer des observations de la classe majoritaire : `Sous-échantillonnage`

Dans notre leçon précédente, nous avons démontré comment traiter les ensembles de données déséquilibrés en utilisant une `recette`. Une recette peut être considérée comme un plan décrivant les étapes à appliquer à un ensemble de données pour le préparer à l'analyse. Dans notre cas, nous souhaitons obtenir une distribution égale du nombre de nos cuisines pour notre `ensemble d'entraînement`. Allons-y directement.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Nous sommes prêts à entraîner des modèles 👩‍💻👨‍💻 !

## 3. Au-delà des modèles de régression multinomiale

Dans notre leçon précédente, nous avons étudié les modèles de régression multinomiale. Explorons maintenant des modèles plus flexibles pour la classification.

### Machines à vecteurs de support

Dans le contexte de la classification, les `Machines à vecteurs de support` sont une technique d'apprentissage automatique qui cherche à trouver un *hyperplan* qui sépare "au mieux" les classes. Prenons un exemple simple :

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ ne sépare pas les classes. H2~ les sépare, mais seulement avec une petite marge. H3~ les sépare avec la marge maximale.

#### Classificateur Linéaire à Vecteurs de Support

Le clustering par vecteurs de support (SVC) est une branche de la famille des machines à vecteurs de support, une technique d'apprentissage automatique. Dans le SVC, l'hyperplan est choisi pour séparer correctement `la plupart` des observations d'entraînement, mais `peut mal classer` quelques observations. En permettant à certains points d'être du mauvais côté, le SVM devient plus robuste face aux valeurs aberrantes, ce qui améliore la généralisation aux nouvelles données. Le paramètre qui régule cette violation est appelé `cost`, avec une valeur par défaut de 1 (voir `help("svm_poly")`).

Créons un SVC linéaire en définissant `degree = 1` dans un modèle SVM polynomial.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Maintenant que nous avons intégré les étapes de prétraitement et la spécification du modèle dans un *workflow*, nous pouvons passer à l'entraînement du SVC linéaire et évaluer les résultats en même temps. Pour les métriques de performance, créons un ensemble de métriques qui évaluera : `accuracy`, `sensitivity`, `Positive Predicted Value` et `F Measure`.

> `augment()` ajoutera une ou plusieurs colonnes pour les prédictions aux données fournies.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Machine à Vecteurs de Support

La machine à vecteurs de support (SVM) est une extension du classificateur à vecteurs de support afin de gérer une frontière non linéaire entre les classes. En substance, les SVM utilisent l’*astuce du noyau* pour agrandir l’espace des caractéristiques et s’adapter aux relations non linéaires entre les classes. Une fonction noyau populaire et extrêmement flexible utilisée par les SVM est la *fonction de base radiale.* Voyons comment elle se comporte avec nos données.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Beaucoup mieux 🤩 !

> ✅ Veuillez consulter :
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning avec R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> pour en savoir plus.

### Classificateurs des plus proches voisins

Le *K*-nearest neighbor (KNN) est un algorithme dans lequel chaque observation est prédite en fonction de sa *similarité* avec d'autres observations.

Essayons d'en ajuster un à nos données.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Il semble que ce modèle ne donne pas de très bons résultats. Modifier les arguments du modèle (voir `help("nearest_neighbor")`) pourrait probablement améliorer ses performances. Assurez-vous d'essayer.

> ✅ Veuillez consulter :
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> pour en savoir plus sur les classificateurs *K*-Nearest Neighbors.

### Classificateurs par ensemble

Les algorithmes d'ensemble fonctionnent en combinant plusieurs estimateurs de base pour produire un modèle optimal, soit en :

`bagging` : appliquant une *fonction de moyenne* à une collection de modèles de base

`boosting` : construisant une séquence de modèles qui s'appuient les uns sur les autres pour améliorer les performances prédictives.

Commençons par essayer un modèle de Forêt Aléatoire, qui construit une grande collection d'arbres de décision, puis applique une fonction de moyenne pour obtenir un meilleur modèle global.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Bon travail 👏 !

Essayons également un modèle Boosted Tree.

Boosted Tree définit une méthode d'ensemble qui crée une série d'arbres de décision séquentiels où chaque arbre dépend des résultats des arbres précédents dans le but de réduire progressivement l'erreur. Il se concentre sur les poids des éléments mal classés et ajuste l'ajustement pour le prochain classificateur afin de corriger.

Il existe différentes façons d'ajuster ce modèle (voir `help("boost_tree")`). Dans cet exemple, nous ajusterons les Boosted Trees via le moteur `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Veuillez consulter :
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Explore le modèle AdaBoost, une bonne alternative à xgboost.
>
> pour en savoir plus sur les classificateurs ensemblistes.

## 4. Extra - comparer plusieurs modèles

Nous avons ajusté un bon nombre de modèles dans ce laboratoire 🙌. Cela peut devenir fastidieux ou compliqué de créer de nombreux workflows à partir de différents ensembles de préprocesseurs et/ou spécifications de modèles, puis de calculer les métriques de performance une par une.

Voyons si nous pouvons résoudre ce problème en créant une fonction qui ajuste une liste de workflows sur l'ensemble d'entraînement, puis retourne les métriques de performance basées sur l'ensemble de test. Nous allons utiliser `map()` et `map_dfr()` du package [purrr](https://purrr.tidyverse.org/) pour appliquer des fonctions à chaque élément d'une liste.

> Les fonctions [`map()`](https://purrr.tidyverse.org/reference/map.html) vous permettent de remplacer de nombreuses boucles for par un code à la fois plus concis et plus facile à lire. Le meilleur endroit pour apprendre les fonctions [`map()`](https://purrr.tidyverse.org/reference/map.html) est le [chapitre sur l'itération](http://r4ds.had.co.nz/iteration.html) dans R for data science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) permet aux utilisateurs de créer et d'ajuster facilement un grand nombre de modèles, mais il est principalement conçu pour fonctionner avec des techniques de rééchantillonnage telles que la `validation croisée`, une approche que nous n'avons pas encore abordée.

## **🚀Défi**

Chacune de ces techniques possède un grand nombre de paramètres que vous pouvez ajuster, comme par exemple `cost` pour les SVM, `neighbors` pour les KNN, ou `mtry` (Prédicteurs Sélectionnés Aléatoirement) pour les Forêts Aléatoires.

Faites des recherches sur les paramètres par défaut de chacun et réfléchissez à ce que modifier ces paramètres pourrait signifier pour la qualité du modèle.

Pour en savoir plus sur un modèle particulier et ses paramètres, utilisez : `help("model")`, par exemple `help("rand_forest")`.

> En pratique, nous *estimons* généralement les *meilleures valeurs* pour ces paramètres en entraînant de nombreux modèles sur un `jeu de données simulé` et en mesurant les performances de tous ces modèles. Ce processus s'appelle **l'optimisation**.

### [**Quiz après le cours**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Révision & Étude Personnelle**

Il y a beaucoup de jargon dans ces leçons, alors prenez un moment pour consulter [cette liste](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) de terminologie utile !

#### MERCI À :

[`Allison Horst`](https://twitter.com/allison_horst/) pour avoir créé les illustrations incroyables qui rendent R plus accueillant et engageant. Retrouvez plus d'illustrations dans sa [galerie](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) et [Jen Looper](https://www.twitter.com/jenlooper) pour avoir créé la version originale en Python de ce module ♥️

Bon apprentissage,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Illustration par @allison_horst</figcaption>



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
